# Deep Learning

In [1]:
import numpy as np
import tensorflow as tf
print("NumPy Version:", np.__version__)
print("TensorFlow Version:", tf.__version__)

NumPy Version: 1.24.3
TensorFlow Version: 2.12.1


In [2]:
import tensorflow as tf
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))
print("Build info:", tf.sysconfig.get_build_info())

Built with CUDA: False
Physical GPUs: []
Build info: OrderedDict([('is_cuda_build', False), ('is_rocm_build', False), ('is_tensorrt_build', False), ('msvcp_dll_names', 'msvcp140.dll,msvcp140_1.dll')])


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

import sys
sys.path.append('..')
from utils import get_train_data, custom_preprocessor, submit_test, train_test_split_temporal


In [4]:
X, y = get_train_data() # X comes out prepared with added columns

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split_temporal(X, y)

X_train = X_train.drop(columns=["date"])
X_valid = X_valid.drop(columns=["date"])


Loading school holidays from cache...


In [5]:
# Fit the preprocessor on the training data
custom_preprocessor.fit(X_train)

# Transform the data
X_train_processed = custom_preprocessor.transform(X_train)
X_valid_processed = custom_preprocessor.transform(X_valid)


In [9]:
import numpy as np
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming X_train, y_train, X_val, y_val are already prepared

# Define a simple model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(1, activation='linear')  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Loss adjusted for regression

# Custom callback to save training and validation scores
class TrainingHistory(Callback):
    def on_train_begin(self, logs=None):
        self.train_scores = []
        self.val_scores = []

    def on_epoch_end(self, epoch, logs=None):
        self.train_scores.append(logs.get('loss'))
        self.val_scores.append(logs.get('val_loss'))

history_callback = TrainingHistory()

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=400,
    batch_size=32,
    callbacks=[history_callback,  ModelCheckpoint("DL_128_64.h5", monitor="val_loss", save_best_only=True)],
    verbose=1
)

Epoch 1/400
14266/14266 [==============================] - 32s 2ms/step - loss: 79.1098 - mae: 2.2377 - val_loss: 2.5646 - val_mae: 1.2560
Epoch 2/400
14266/14266 [==============================] - 32s 2ms/step - loss: 2.9066 - mae: 1.4265 - val_loss: 2.2369 - val_mae: 1.2572
Epoch 3/400
14266/14266 [==============================] - 29s 2ms/step - loss: 2.8042 - mae: 1.4255 - val_loss: 2.2121 - val_mae: 1.2495
Epoch 4/400
14266/14266 [==============================] - 29s 2ms/step - loss: 2.8034 - mae: 1.4254 - val_loss: 2.2248 - val_mae: 1.2544
Epoch 5/400
14266/14266 [==============================] - 29s 2ms/step - loss: 2.8034 - mae: 1.4255 - val_loss: 2.2259 - val_mae: 1.2548
Epoch 6/400
14266/14266 [==============================] - 29s 2ms/step - loss: 2.8034 - mae: 1.4255 - val_loss: 2.2276 - val_mae: 1.2554
Epoch 7/400
14266/14266 [==============================] - 30s 2ms/step - loss: 2.8034 - mae: 1.4255 - val_loss: 2.2161 - val_mae: 1.2510
Epoch 8/400
14266/14266 [========

In [ ]:
# Save the scores for later plotting
np.save("train_scores.npy", np.array(history_callback.train_scores))
np.save("val_scores.npy", np.array(history_callback.val_scores))

In [ ]:
model.save("DL_128_64.h5")

In [10]:
y_pred = model.predict(X_valid)

score = np.sqrt(mean_squared_error(y_pred, y_valid))

1260/1260 [==============================] - 3s 2ms/step


In [11]:
np.round(score, 2)

1.49

In [ ]:
from utils import preparation
file_name = "../submissions/" + "deep_learning_initial" + "_submission.csv"

X_test = pd.read_parquet("../data/final_test.parquet")
X_test = preparation(X_test)
X_test = X_test.drop(columns=["date"])
X_test_processed = custom_preprocessor.transform(X_test)

y_predict = model.predict(X_test_processed)

results = pd.DataFrame(
dict(
    Id=np.arange(y_predict.shape[0]),
    log_bike_count=y_predict.flatten(),
    )
)
results.to_csv(file_name, index=False)

Loading school holidays from cache...
1608/1608 [==============================] - 3s 2ms/step


In [ ]:
#submit_test(pipeline, "Ridge")